# Quiz Question Correctness Evaluation

Evaluating the MongoDB quiz question dataset using Galileo modules.

Galileo platform - https://console.acme.rungalileo.io/ 

In [1]:
!pip3 install promptquality pymongo uuid


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# load data from mongodb
import ipython_secrets
from pymongo import MongoClient
from bson import ObjectId

MONGODB_CONNECTION_URI = ipython_secrets.get_secret('MONGODB_CONNECTION_URI')

client = MongoClient(MONGODB_CONNECTION_URI)
db = client["docs-chatbot-dev"]
generated_data_collection = db["generated_data"]
gpt_35_quiz_questions = list(generated_data_collection.find({ 'commandRunId': ObjectId('66a2832f1b5f11cf652b9228') }))
gpt_4o_quiz_questions = list(generated_data_collection.find({ 'commandRunId': ObjectId('66a2832f1b5f11cf652b922a') }))
print(f"Got # GPT-3.5 quiz questions: {len(gpt_35_quiz_questions)}")
print(f"Got # GPT-4o quiz questions: {len(gpt_4o_quiz_questions)}")
client.close()

Got # GPT-3.5 quiz questions: 466
Got # GPT-4o quiz questions: 467


In [5]:
from pprint import pprint
pprint(gpt_35_quiz_questions[0]

{'_id': ObjectId('66a2833c1b5f11cf652b9249'),
 'commandRunId': ObjectId('66a2832f1b5f11cf652b9228'),
 'createdAt': datetime.datetime(2024, 7, 25, 16, 54, 20, 905000),
 'data': {'modelAnswer': 'A,B,D'},
 'evalData': {'_id': ObjectId('6669edf97da46f49f235f083'),
              'answers': [{'answer': 'It eliminates data duplication.',
                           'isCorrect': False,
                           'label': 'A'},
                          {'answer': 'It reduces document size by moving  data '
                                     'that is not frequently used to related '
                                     'documents.',
                           'isCorrect': True,
                           'label': 'B'},
                          {'answer': 'It is useful for large documents that '
                                     'contain long lists of items in an array.',
                           'isCorrect': True,
                           'label': 'C'},
                          {'answ

In [4]:
import promptquality as pq

pq.login("https://console.acme.rungalileo.io/")

Go to https://console.acme.rungalileo.io/get-token to generate a new Galileo token.


🔐 Enter your Galileo token: ········


👋 You have logged into 🔭 Galileo (https://console.acme.rungalileo.io/) as ben.p@mongodb.com.


Config(console_url=Url('https://console.acme.rungalileo.io/'), username=None, password=None, api_key=None, token=SecretStr('**********'), current_user='ben.p@mongodb.com', current_project_id=None, current_project_name=None, current_run_id=None, current_run_name=None, current_run_url=None, current_run_task_type=None, current_template_id=None, current_template_name=None, current_template_version_id=None, current_template_version=None, current_template=None, current_dataset_id=None, current_job_id=None, current_prompt_optimization_job_id=None, api_url=Url('https://api.acme.rungalileo.io/'))

In [28]:
from promptquality import NodeType, NodeRow
from uuid import uuid4
from pprint import pprint

def extract_eval_data_from_quiz_gen_data(quiz_gen_data):
    question = quiz_gen_data["evalData"]["promptMessages"][-1]["content"]
    model_answer = quiz_gen_data["data"]["modelAnswer"]
    correct_answer = create_correct_answer_str(quiz_gen_data)
    return {
        "question": question,
        "system_output": model_answer,
        "correct_answer": correct_answer
    }

def create_correct_answer_str(quiz_gen_data):
    """
    Create a string representation of the correct answer, sorted alphabetically with multiple correct answers (if they exist), comma-separated.
    e.g.
    A
    A,C
    A,B,C,D
    etc
    """
    # Extract answers from evalData
    answers = quiz_gen_data.get('evalData', {}).get('answers', [])
    
    # Filter out the correct answers
    correct_answers = [answer['label'] for answer in answers if answer['isCorrect']]
    
    # Sort the correct answers alphabetically
    correct_answers.sort()
    
    # Join the sorted answers with commas
    correct_answer_str = ','.join(correct_answers)
    
    return correct_answer_str

def create_nodes_for_generated_data(generated_data):
    eval_data = extract_eval_data_from_quiz_gen_data(generated_data)
    rows = []
    CHAIN_ROOT_ID = uuid4()
    question = eval_data["question"]
    system_output = eval_data["system_output"]
    correct_answer = eval_data["correct_answer"]
    step = 0
    # create root node
    rows.append(
        NodeRow(
            node_id=CHAIN_ROOT_ID,
            chain_root_id=CHAIN_ROOT_ID, # UUID of the 'parent' node
            step = step, #an integer indicating which step this node is
            node_input=question, # input into your overall sequence or chain
            node_output=system_output, # output of your overall sequence or chain
            # TODO: add golden answer...
            node_type=NodeType.llm, # Can be chain, retriever, llm, chat, agent, tool
            target=correct_answer
        )
    )
    return rows

pprint(create_nodes_for_generated_data(gpt_35_quiz_questions[60]))

[NodeRow(node_id=UUID('9f8aa344-00a0-4f14-8a83-79e0f8c850e5'), node_type=<NodeType.llm: 'llm'>, node_name=None, node_input="Question: Which of the following is something that you can do with MongoDB Atlas? (Select all that apply.)\nA. Write and host a full application in a managed cloud environment.\nB. Store your data with MongoDB's multi-cloud, global service.\nC. Query across multiple Atlas clusters to get a holistic view of your data.\nD. Add search functionality to your application, like a search bar.\nE. None of the above.\nResponse: ", node_output='A,B,C,D', chain_root_id=UUID('9f8aa344-00a0-4f14-8a83-79e0f8c850e5'), chain_id=None, step=0, has_children=False, inputs={}, prompt=None, response=None, creation_timestamp=1723052402483120000, finish_reason='', latency=None, query_input_tokens=0, query_output_tokens=0, query_total_tokens=0, params={}, target='A,B,C,D')]


In [36]:
# Create custom scorer to check if regex match /mongodb/i
import re 
def executor(row: NodeRow) -> float:
    return 1.0 if row.node_output == row.target else 0.0


def aggregator(scores, indices) -> dict:
    return {
        '% Correct': sum(scores)/len(scores)
    }

is_correct = pq.CustomScorer(name='Question Correct', executor=executor, aggregator=aggregator)

for gd in gpt_35_quiz_questions[0:5]:
    print(executor(create_nodes_for_generated_data(gd)[0]))

0.0
1.0
1.0
1.0
1.0


In [35]:
from datetime import datetime, timezone
def create_all_nodes_for_gen_data(gen_data_lst):
    
    all_the_nodes = []
    for gen_data in gen_data_lst:
        nodes = create_nodes_for_generated_data(gen_data)
        all_the_nodes += nodes
    return all_the_nodes

runs = [
    ("gpt-3.5-turbo", gpt_35_quiz_questions),
    ("gpt-4o", gpt_4o_quiz_questions)
]

project_name = "quiz_questions"
date_time_str = datetime.now(timezone.utc).strftime("%Y-%m-%d_%H:%M:%S")
for (run_name, gen_data) in runs:
    nodes = create_all_nodes_for_gen_data(gen_data)
    pq.chain_run(
        nodes,
        project_name=project_name,
        scorers=[
            is_correct
        ],
        run_name=f"{run_name}__{date_time_str}"
    )
        

Go to https://console.acme.rungalileo.io/get-token to generate a new Galileo token.


🔐 Enter your Galileo token: ········


👋 You have logged into 🔭 Galileo (https://console.acme.rungalileo.io/) as ben.p@mongodb.com.


Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
rouge: Done ✅
cost: Done ✅
bleu: Done ✅
toxicity: Computing 🚧
pii: Computing 🚧
protect_status: Done ✅
latency: Done ✅
🔭 View your prompt run on the Galileo console at: https://console.acme.rungalileo.io/prompt/chains/31ab429a-4f94-4059-9b49-edc0bea12783/dceab7d5-1a79-43d9-94a2-9d8786462238?taskType=12
output: A,B,D
target: B,C,D
output: A,C
target: A,C
output: A
target: A
output: A
target: A
output: B
target: B
output: B,C,D
target: B,C
output: A,C
target: A,C
output: B
target: B
output: A
target: A
output: B
target: B
output: A
target: A
output: C
target: C
output: A,C,D
target: A,C,D
output: B
target: B
output: C
target: C,D
output: B,C
target: B,C
output: C
target: A,B
output: A
target: A
output: A,C,D
target: A,D
output: D
target: D
output: B
target: B
output: A
target: A
output: B
target: B
output: C
target: C
output: C
target: C
output: B,C
target: B,C
output: A
target: A
output: B,C,D
target: B,C
output: D
t

Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
rouge: Done ✅
cost: Done ✅
bleu: Done ✅
toxicity: Computing 🚧
pii: Computing 🚧
protect_status: Done ✅
latency: Done ✅
🔭 View your prompt run on the Galileo console at: https://console.acme.rungalileo.io/prompt/chains/31ab429a-4f94-4059-9b49-edc0bea12783/eeea230c-80b5-4ec2-a3fc-a756fbffe118?taskType=12
output: A,B,C,D
target: A,B,C,D
output: B,C,D
target: B,C,D
output: A
target: A
output: B,C,D
target: B,C
output: A,C
target: A,C
output: A,B,C
target: A,B
output: C
target: C
output: A,C
target: A,C
output: A,C
target: A,C
output: C
target: C
output: A,B,C
target: A,B,C
output: B,C
target: B,C
output: A,B
target: A,B
output: B,C
target: B,C
output: A,D
target: A,D
output: A
target: A
output: B
target: B
output: A,B,C
target: A,B
output: B
target: B
output: C
target: C
output: A
target: A
output: A
target: A
output: B
target: B
output: C
target: C
output: A,B,C,D
target: B,D
output: A,C,D
target: A,C,D
output: D
targe